In [1]:
# Standard imports

# get the environment
import sys
print(sys.version)
print(sys.executable)

3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
C:\Users\bbutler\Documents\Anaconda3\envs\timeseries\python.exe


In [2]:
# import base libraries
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import os

# set options
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
# set the directory
loanDir = 'C:\\Users\\bbutler\\Documents\\PLoan'

os.chdir(loanDir)

# get the file
# bring in balances

ploanPort = pd.read_csv('TestPloanV2.csv',low_memory=False)
ploanPort.head()

,Id,ContractAmt,Term,APR
0,1,3000,48,0.04
1,2,9824,48,0.10
2,3,100000,360,0.06
3,4,3000,48,0.04


In [4]:
ploanPort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           4 non-null      int64  
 1   ContractAmt  4 non-null      int64  
 2   Term         4 non-null      int64  
 3   APR          4 non-null      float64
dtypes: float64(1), int64(3)
memory usage: 256.0 bytes


In [5]:
# formula to calculate the payment
def calcPayment(origBalance, term, apr):
    """
    Function calculates the monthly payment for a Loan given the balance, term and apr
    
    term: should be a period in months
    
    
    """
    # break formula into numerator and denominator
    numerator = (origBalance * (apr/12) * (1+(apr/12))**term)
    denominator = (((1+(apr/12))**term) - 1)
    
    monthPayment = numerator/denominator
    
    return monthPayment

In [6]:
calcPayment(9824, 48, .1033)

250.72181872363586

In [7]:
# run a test of monthly payment to compare

ploanPort['CalcPayment'] = ploanPort.apply(lambda df: calcPayment(df.ContractAmt, df.Term, df.APR), axis = 1)
ploanPort.head()

,Id,ContractAmt,Term,APR,CalcPayment
0,1,3000,48,0.04,67.74
1,2,9824,48,0.10,250.72
2,3,100000,360,0.06,599.55
3,4,3000,48,0.04,67.74


In [8]:
# pull the values from the ploanPort

monthly = ploanPort['CalcPayment'][0]
orig = ploanPort['ContractAmt'][0]
term = ploanPort['Term'][0]
rate = ploanPort['APR'][0]

# inputs for loan costs
costOfFunds = .025
provisions = .01
servicingCosts = 13.34
nplRate = .0025
originationCosts = 54.72
earlyPayoff = .0043
docSupport = 19.56
discountRate = .10

In [9]:
ploanPort['Interest'] = (ploanPort['CalcPayment']*ploanPort['Term']) - ploanPort['ContractAmt']
ploanPort.head()

,Id,ContractAmt,Term,APR,CalcPayment,Interest
0,1,3000,48,0.04,67.74,251.38
1,2,9824,48,0.10,250.72,2210.65
2,3,100000,360,0.06,599.55,115838.19
3,4,3000,48,0.04,67.74,251.38


In [10]:
# populate period for first loan

# set the length
term = 48

periods = np.array(range(0,term + 1))
periods

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48])

In [11]:
# create the empty data frame

dict = {'Period': periods}

loanDf = pd.DataFrame(dict)
loanDf.head()

,Period
0,0
1,1
2,2
3,3
4,4


In [12]:
loanDf['MonthlyPayment'] = monthly
loanDf.head()

,Period,MonthlyPayment
0,0,67.74
1,1,67.74
2,2,67.74
3,3,67.74
4,4,67.74


In [13]:
# zero out monthly payment for first month
loanDf.loc[0, 'MonthlyPayment'] = 0

In [14]:
loanDf['OrigBal'] = orig
loanDf.head()

,Period,MonthlyPayment,OrigBal
0,0,0.00,3000
1,1,67.74,3000
2,2,67.74,3000
3,3,67.74,3000
4,4,67.74,3000


In [15]:
# now iterate through like the following
# for i in range(1, len(df)):
#    df.loc[i, 'C'] = df.loc[i-1, 'C'] * df.loc[i, 'A'] + df.loc[i, 'B']
loanDf.loc[0, 'OutBal'] = loanDf.loc[0, 'OrigBal'] - loanDf.loc[0, 'MonthlyPayment']

for i in range(1, len(loanDf)):
    loanDf.loc[i, 'OrigBal'] = loanDf.loc[i - 1, 'OutBal']
    loanDf.loc[i, 'OutBal'] = loanDf.loc[i, 'OrigBal'] * (1 + rate/12) - loanDf.loc[i, 'MonthlyPayment']
    
# then drop row zero
loanDf = loanDf.drop(0)    
    
    
loanDf.head(10)

,Period,MonthlyPayment,OrigBal,OutBal
1,1,67.74,3000.00,2942.26
2,2,67.74,2942.26,2884.33
3,3,67.74,2884.33,2826.21
4,4,67.74,2826.21,2767.89
5,5,67.74,2767.89,2709.38
6,6,67.74,2709.38,2650.68
7,7,67.74,2650.68,2591.78
8,8,67.74,2591.78,2532.68
9,9,67.74,2532.68,2473.38
10,10,67.74,2473.38,2413.89


In [16]:
# create interest income
loanDf['IntIncome'] = (loanDf['OrigBal']*(1+rate/12))-loanDf['OrigBal']
loanDf.head()

,Period,MonthlyPayment,OrigBal,OutBal,IntIncome
1,1,67.74,3000.00,2942.26,10.00
2,2,67.74,2942.26,2884.33,9.81
3,3,67.74,2884.33,2826.21,9.61
4,4,67.74,2826.21,2767.89,9.42
5,5,67.74,2767.89,2709.38,9.23


In [17]:
# create all of the cost columns from the inputs

loanDf['CostOfFunds'] = loanDf['OutBal'] * costOfFunds/12
loanDf['Provisions'] = loanDf['OutBal'] * provisions/12
loanDf['ServicingCosts'] = loanDf['Period'].apply(lambda x: servicingCosts if x < term else 0)
loanDf['OriginationCosts'] = originationCosts/term
loanDf['DocSupport'] = docSupport/term
loanDf['EarlyPayment'] = loanDf['OutBal'] * earlyPayoff/12

loanDf

,Period,MonthlyPayment,OrigBal,OutBal,IntIncome,CostOfFunds,Provisions,ServicingCosts,OriginationCosts,DocSupport,EarlyPayment
1,1,67.74,3000.00,2942.26,10.00,6.13,2.45,13.34,1.14,0.41,1.05
2,2,67.74,2942.26,2884.33,9.81,6.01,2.40,13.34,1.14,0.41,1.03
3,3,67.74,2884.33,2826.21,9.61,5.89,2.36,13.34,1.14,0.41,1.01
4,4,67.74,2826.21,2767.89,9.42,5.77,2.31,13.34,1.14,0.41,0.99
5,5,67.74,2767.89,2709.38,9.23,5.64,2.26,13.34,1.14,0.41,0.97
6,6,67.74,2709.38,2650.68,9.03,5.52,2.21,13.34,1.14,0.41,0.95
7,7,67.74,2650.68,2591.78,8.84,5.40,2.16,13.34,1.14,0.41,0.93
8,8,67.74,2591.78,2532.68,8.64,5.28,2.11,13.34,1.14,0.41,0.91
9,9,67.74,2532.68,2473.38,8.44,5.15,2.06,13.34,1.14,0.41,0.89
10,10,67.74,2473.38,2413.89,8.24,5.03,2.01,13.34,1.14,0.41,0.86


In [18]:
# select the columns to add up for costs

costCols = ['CostOfFunds', 'Provisions', 'ServicingCosts',
         'OriginationCosts', 'DocSupport', 'EarlyPayment']

In [19]:
# make a total 
#df['Fruit Total']= df.iloc[:, -4:-1].sum(axis=1)

loanDf['TotalCost'] = loanDf.loc[:,costCols].sum(axis=1)
loanDf['Profit'] = loanDf['IntIncome'] - loanDf['TotalCost']
loanDf['NPV'] = loanDf['Profit']/((1+discountRate/12)**loanDf['Period'])
loanDf.head()

,Period,MonthlyPayment,OrigBal,OutBal,IntIncome,CostOfFunds,Provisions,ServicingCosts,OriginationCosts,DocSupport,EarlyPayment,TotalCost,Profit,NPV
1,1,67.74,3000.00,2942.26,10.00,6.13,2.45,13.34,1.14,0.41,1.05,24.52,-14.52,-14.40
2,2,67.74,2942.26,2884.33,9.81,6.01,2.40,13.34,1.14,0.41,1.03,24.33,-14.53,-14.29
3,3,67.74,2884.33,2826.21,9.61,5.89,2.36,13.34,1.14,0.41,1.01,24.14,-14.53,-14.17
4,4,67.74,2826.21,2767.89,9.42,5.77,2.31,13.34,1.14,0.41,0.99,23.95,-14.53,-14.06
5,5,67.74,2767.89,2709.38,9.23,5.64,2.26,13.34,1.14,0.41,0.97,23.76,-14.53,-13.94


In [20]:
loanDf

,Period,MonthlyPayment,OrigBal,OutBal,IntIncome,CostOfFunds,Provisions,ServicingCosts,OriginationCosts,DocSupport,EarlyPayment,TotalCost,Profit,NPV
1,1,67.74,3000.00,2942.26,10.00,6.13,2.45,13.34,1.14,0.41,1.05,24.52,-14.52,-14.40
2,2,67.74,2942.26,2884.33,9.81,6.01,2.40,13.34,1.14,0.41,1.03,24.33,-14.53,-14.29
3,3,67.74,2884.33,2826.21,9.61,5.89,2.36,13.34,1.14,0.41,1.01,24.14,-14.53,-14.17
4,4,67.74,2826.21,2767.89,9.42,5.77,2.31,13.34,1.14,0.41,0.99,23.95,-14.53,-14.06
5,5,67.74,2767.89,2709.38,9.23,5.64,2.26,13.34,1.14,0.41,0.97,23.76,-14.53,-13.94
6,6,67.74,2709.38,2650.68,9.03,5.52,2.21,13.34,1.14,0.41,0.95,23.57,-14.54,-13.83
7,7,67.74,2650.68,2591.78,8.84,5.40,2.16,13.34,1.14,0.41,0.93,23.38,-14.54,-13.72
8,8,67.74,2591.78,2532.68,8.64,5.28,2.11,13.34,1.14,0.41,0.91,23.18,-14.54,-13.61
9,9,67.74,2532.68,2473.38,8.44,5.15,2.06,13.34,1.14,0.41,0.89,22.99,-14.55,-13.50
10,10,67.74,2473.38,2413.89,8.24,5.03,2.01,13.34,1.14,0.41,0.86,22.79,-14.55,-13.39


In [21]:
# roll it all up

loanTotal = loanDf.groupby(['MonthlyPayment'],
                        as_index=False).agg({'IntIncome': np.sum,
                                             'TotalCost': np.sum,
                                             'Profit': np.sum,
                                             'NPV': np.sum})

loanTotal.head()

,MonthlyPayment,IntIncome,TotalCost,Profit,NPV
0,67.74,251.38,938.42,-687.04,-566.16


In [22]:
# set the columns to append

cols = loanTotal.columns[2:].values.tolist()
cols

['TotalCost', 'Profit', 'NPV']

In [23]:
ploanComp = pd.concat([ploanPort,pd.DataFrame(columns=list(cols))])
ploanComp.head()

,Id,ContractAmt,Term,APR,CalcPayment,Interest,TotalCost,Profit,NPV
0,1.00,3000.00,48.00,0.04,67.74,251.38,NaN,NaN,NaN
1,2.00,9824.00,48.00,0.10,250.72,2210.65,NaN,NaN,NaN
2,3.00,100000.00,360.00,0.06,599.55,115838.19,NaN,NaN,NaN
3,4.00,3000.00,48.00,0.04,67.74,251.38,NaN,NaN,NaN


In [24]:
# put this in the porfolio DB

j = 1

for i in range(2,5):
    ploanComp.iloc[0,j+5] = loanTotal.iloc[0,i]
    j += 1
    
ploanComp.head()

,Id,ContractAmt,Term,APR,CalcPayment,Interest,TotalCost,Profit,NPV
0,1.00,3000.00,48.00,0.04,67.74,251.38,938.42,-687.04,-566.16
1,2.00,9824.00,48.00,0.10,250.72,2210.65,NaN,NaN,NaN
2,3.00,100000.00,360.00,0.06,599.55,115838.19,NaN,NaN,NaN
3,4.00,3000.00,48.00,0.04,67.74,251.38,NaN,NaN,NaN
